# Vertex AI Agent SDK Example - Company News Agent Application

## Overview

Vertex AI Agent SDK is a platform for creating and managing GenAI applications with Agents, that can use function and extension based tools to connect large language models to external systems via APIs.

These external systems can provide LLMs with real-time data and perform data processing actions on their behalf. You can use pre-built (e.g. code-interpreter) or your own extensions in Vertex AI Agents SDK.

<!-- Learn more about [Vertex AI Extensions](https://cloud.google.com/vertex-ai/generative-ai/docs/extensions/overview).-->


### Objective

In this example, you learn how to create an extension service backend on Cloud Run, register the extension with Vertex AI Agent SDK, and then use the Agent Application in a user session.

You can answer of complex questions that combine LLM language skills with the data from the extension, allowing for multiple chained retrievals driven by the LLM.

The steps performed include:

- The Neo4j Company News database
- Creating the extension service running on Cloud Run
- Creating an OpenAPI 3.1 YAML file for the Cloud Run service
- Creating an Vertex AI Agent SDK application
- Registering the service as an extension for an agent with Vertex AI Agent SDK
- Using a session to respond to user complex user queries which will use a number of agent calls behind the scenes


## How it works

This notebook provides an external Neo4j Extension to be used with the Agent of our application.

The extension makes a number of REST endpoints available for the agent to query a company news graph with articles about companies their industries and executives involved with the companies.

It is deployed as a Flask app in a Docker container with Google Cloud run.

This guide assumes that you are somewhat familiar with

* [Vertex AI Agent SDK](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-api)
* [LangChain](https://python.langchain.com/docs/get_started/introduction)
* [OpenAPI specification](https://swagger.io/specification/)
* [Cloud Run](https://cloud.google.com/run/docs)

**_NOTE_**:

Your account needs to have permissions and API enabled for Vertex AI and Cloud Run.

The notebook has been tested in the following environment: Python version = 3.11

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).
1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).
1. Your project must also be allowlisted for the Vertex AI Extension Private Preview.
1. This notebook requires that you have the following permissions for your GCP project:
- `roles/aiplatform.user`

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

##Authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()

##Download most recent copy of Vertex Agents SDK

In [ ]:
!gsutil cp gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.61.dev20240814+vertex.agents-py2.py3-none-any.whl .
!pip install --quiet --upgrade --force-reinstall -q google_cloud_aiplatform-1.61.dev20240814+vertex.agents-py2.py3-none-any.whl --no-warn-conflicts
!pip install --quiet -U "pandas==2.2.2"
!pip install --quiet -U 'numpy<2'

# Restart the kernel runtime to load the private preview SDK
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Copying gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.61.dev20240814+vertex.agents-py2.py3-none-any.whl...
\ [1 files][  5.3 MiB/  5.3 MiB]                                                
Operation completed over 1 objects/5.3 MiB.                                      
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 15

{'status': 'ok', 'restart': True}

##

##Imports and Initialization


Make sure to initialize aiplatform with your projectID, location, and api endpoint. You need to initialize aiplatform before performing any of the other imports.


In [ ]:
PROJECT_ID = "vertex-ai-neo4j-extension" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type: "string"}
API_ENDPOINT = 'us-central1-aiplatform.googleapis.com'

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, api_endpoint=API_ENDPOINT)

## Neo4j Company News Extension

This is the Flask application, connecting to Neo4j and hosting 6 different endpoints to retrieve information about companies, article, people, industries etc.

### Endpoints

Each endpoint takes parameters via query string and returns it's response as JSON

* `/industries` - List of Industry names
* `/companies` - List of Companies (id, name, summary) by fulltext `search`
* `/companies_in_industry` - Companies (id, name, summary) in a given industry by `industry`
* `/articles_in_month` - List of Articles (id, author, title, date, sentiment) in a month timeframe from the given `date` (yyyy-mm-dd)
* `/article` - Single Article details (id, author, title, date, sentiment, site, summary, content) by article `id`
*  `/companies_in_articles` - Companies (id, name, summary) mentioned in articles by list of article `ids`
* `/people_at_company` - People (name, role) associated with a company by company `id`

It uses a publicly hosted dataset with read-only credentials for the "companies" graph.

In [ ]:
# Connection Details

NEO4J_URI='neo4j+s://demo.neo4jlabs.com'
NEO4J_USERNAME='companies'
NEO4J_PASSWORD='companies'
NEO4J_DATABASE='companies'



In [ ]:
import os
if not os.path.exists("extension"):
    os.mkdir("extension")

### Flask Application

In [ ]:
%%writefile extension/extension.py
import os
from flask import Flask, jsonify, request
from neo4j import GraphDatabase

app = Flask(__name__)

URI = os.getenv('NEO4J_URI', 'neo4j+s://demo.neo4jlabs.com')
AUTH = (os.getenv('NEO4J_USERNAME','companies'),os.getenv('NEO4J_PASSWORD','companies'))

driver = GraphDatabase.driver(URI, auth = AUTH)

@app.route("/industries", methods=["GET"])
def industries():
    """
    List of Industry names
    """

    query = """
    MATCH (i:IndustryCategory) RETURN i.name as industry
    """
    records, _, _ = driver.execute_query(query, _database="companies")
    results = [r['industry'] for r in records]
    return jsonify({ "output": results })

@app.route("/companies", methods=["GET"])
def companies():
    """
    List of Companies (id, name, summary) by fulltext search
    """

    query = """
    CALL db.index.fulltext.queryNodes('entity', $search, {limit: 25})
    YIELD node as c, score WHERE c:Organization
    AND not exists { (c)<-[:HAS_SUBSIDARY]-() }
    RETURN c.id as id, c.name as name, c.summary as summary
    """
    args = request.args
    search = args.get("search")
    records, _, _ = driver.execute_query(query, search=search, _database="companies")
    results = [{"Company": r['name'], "Id": r['id'], "Summary":r['summary']} for r in records]
    return jsonify({ "output": results })

@app.route("/companies_in_industry", methods=["GET"])
def companies_in_industry():
    """
    Companies (id, name, summary) in a given industry by name
    """
    query = """
    MATCH (:IndustryCategory {name:$industry})<-[:HAS_CATEGORY]-(c)
    WHERE not exists { (c)<-[:HAS_SUBSIDARY]-() }
    RETURN c.id as id, c.name as name, c.summary as summary
    """
    args = request.args
    industry = args.get("industry")
    records, _, _ = driver.execute_query(query, industry=industry, _database="companies")
    results = [{"Company": r['name'], "Id": r['id'], "Summary":r['summary']} for r in records]
    return jsonify({ "output": results })

@app.route("/articles_in_month", methods=["GET"])
def articles_in_month():
    """
    List of Articles (id, author, title, date, sentiment) in a month timeframe from the given date
    """
    query = """
    match (a:Article)
    where date($date) <= date(a.date)  < date($date) + duration('P1M')
    return a.id as id, a.author as author, a.title as title, toString(a.date) as date, a.sentiment as sentiment
    limit 25
    """
    args = request.args
    date = args.get("date")

    records, _, _ = driver.execute_query(query, date=date, _database="companies")
    results = [{"Article": r['title'], "Id": r['id'], "Date": r['date'], "Sentiment":r['sentiment'], "Author":r['author']} for r in records]
    return jsonify({ "output": results })

@app.route("/article", methods=["GET"])
def article():
    """
    Single Article details (id, author, title, date, sentiment, site, summary, content) by article id
    """

    query = """
    match (a:Article)-[:HAS_CHUNK]->(c:Chunk)
    where a.id = $id
    with a, c order by id(c) asc
    with a, collect(c.text) as content
    return a.id as id, a.author as author, a.title as title, toString(a.date) as date,
    a.summary as summary, a.siteName as site, a.sentiment as sentiment, content
    """
    args = request.args
    id = args.get("id")

    records, _, _ = driver.execute_query(query, id=id, _database="companies")
    results = [{"Article": r['title'], "Id": r['id'], "Date": r['date'], "Sentiment":r['sentiment'], "Author":r['author'],
                "Site": r['site'], "Summary": r['summary'], "Content": r['content']} for r in records]
    return jsonify({ "output": results })

@app.route("/companies_in_articles", methods=["GET"])
def companies_in_articles():
    """
    Companies (id, name, summary) mentioned in articles by list of article ids
    """

    query = """
    MATCH (a:Article)-[:MENTIONS]->(c)
    WHERE a.id in $ids AND not exists { (c)<-[:HAS_SUBSIDARY]-() }
    RETURN c.id as id, c.name as name, c.summary as summary
    """
    args = request.args
    ids = args.get("ids").split(",")

    records, _, _ = driver.execute_query(query, ids=ids, _database="companies")
    results = [{"Company": r['name'], "Id": r['id'], "Summary":r['summary']} for r in records]
    return jsonify({ "output": results })


@app.route("/people_at_company", methods=["GET"])
def people_at_company():
    """
    People (id, name, summary) associated with a company by company id
    """

    query = """
    MATCH (c:Organization)-[role]-(p:Person) WHERE c.id = $id
    RETURN replace(type(role),"HAS_","") as role, p.name as name
    """
    args = request.args
    id = args.get("id")

    records, _, _ = driver.execute_query(query, id=id, _database="companies")
    results = [{"Person": r['name'], "Role": r['role']} for r in records]
    return jsonify({ "output": results })

if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port=int(os.environ.get("PORT", 8080)))

In [ ]:
%%writefile extension/requirements.txt
Flask==2.3.3
gunicorn==23.0.0
neo4j==5.22.0

In [ ]:
%%writefile extension/Dockerfile

FROM python:3.11-slim

ENV PYTHONUNBUFFERED=True

ENV APP_HOME=/app
WORKDIR $APP_HOME
COPY . ./

RUN pip install --no-cache-dir -r requirements.txt

CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 --timeout 0 extension:app

In [ ]:
%%writefile extension/.dockerignore
Dockerfile
README.md
*.pyc
*.pyo
*.pyd
__pycache__
.pytest_cache

Next, you deploy the service to Cloud Run. However, you might need to log in once more to deploy.

In [ ]:
!gcloud auth login

In [ ]:
!gcloud run deploy extension --region=us-central1 --allow-unauthenticated --source extension --no-user-output-enabled

List the most recent Cloud Run service that was deployed, then you'll copy its URL to the next cell:

In [ ]:
!gcloud run services list | sort -k 3 | head -2

Copy paste the output from the previous command here

In [ ]:
# @title Copy paste the output from the previous command here
service_url = "https://extension-wd2kzf73yq-uc.a.run.app/"  # @param {type:"string"}

### Test the deployed service

First, check that your service can accept simple HTTP `GET` requests:

In [ ]:
import requests

url = f'{service_url}/companies_in_industry?industry=Electronic%20Products%20Manufacturers'

r = requests.get(url, headers={ 'Accept': 'application/json' })

print(f"Status Code: {r.status_code}, Content: {r.text}")


Status Code: 200, Content: {"output":[{"Company":"Altium","Id":"EJH622s4BMu2i04Ye7J6fxQ","Summary":"Software company"},{"Company":"Northrop Grumman","Id":"EXjyKMX-3MEG_BMYe08uKtg","Summary":null},{"Company":"Microsemi","Id":"EgtXOU4GjMSuesdkX0nz_Qg","Summary":"Communications corporation"},{"Company":"Nice Systems","Id":"EsLssnhntPTqbf_OUYWJ5oQ","Summary":null},{"Company":"Skyworks Solutions","Id":"EzY1DUFnTNYang87g0sX-2Q","Summary":"American semiconductor manufacturer"},{"Company":"Murrietta Circuits","Id":"Euff3XPIoPkGSqQJSwxk8wg","Summary":null},{"Company":"Airwolf 3D","Id":"E8Hvn4tyBOmGaT1zWRMKlJw","Summary":null},{"Company":"Xi'an System Sensor Electronics","Id":"EUdCZjJ7dOYOVFy9CcmAfvQ","Summary":"Manufacturing company based in Long\u2019an Xian, Guangxi Zhuang Autonomous Region, China and owned by Honeywell"},{"Company":"Bunker Ramo","Id":"EuQUgt5BHPUKqaOG8nrXBcg","Summary":"Manufacturing company owned by Honeywell founded in 1964"},{"Company":"Intermec","Id":"Eb7CrOtCJMQ-Mm5VePc

### Create an OpenAPI spec

Your Vertex Extension requires an OpenAPI 3.1 YAML file that defines routes, URL, HTTP methods, requests, and responses from your "backend" service. The following code creates a YAML file that you need to upload to your Cloud Storage bucket.

In [ ]:
if not os.path.exists("extension-api"):
    os.mkdir("extension-api")


### OpenAPI 3.1.0 YAML Spec for the Neo4j Company Service

In [ ]:
openapi_yaml=f"""
openapi: "3.1.0"
info:
  version: 1.0.0
  title: neo4j_extension
  description: Service for company news information, industries, articles, people at the companies and more.
servers:
  - url: {service_url}
paths:
  /companies:
    get:
      operationId: companies
      description: List of Companies (id, name, summary) by fulltext search
      parameters:
        - name: search
          in: query
          description: Part of a name of a company to search for
          required: true
          schema:
            type: string
      responses:
        '200':
          description: Returns List of Companies (id, name, summary) as JSON
          content:
            application/json:
             schema:
                type: array
                items:
                  type: object
                  properties:
                    id:
                      type: string
                    name:
                      type: string
                    summary:
                      type: string
  /industries:
    get:
      operationId: industries
      description: List of Industry names
      responses:
        '200':
          description: Returns List of Industry Names as JSON
          content:
            application/json:
             schema:
                type: array
                items:
                  type: string
  /companies_in_industry:
    get:
      operationId: companies_in_industry
      description: List of Companies (id, name, summary) in a certain industry by name
      parameters:
        - name: industry
          in: query
          description: Exact industry name from the industries operation
          required: true
          schema:
            type: string
      responses:
        '200':
          description: Returns List of Companies (id, name, summary) as JSON
          content:
            application/json:
             schema:
                type: array
                items:
                  type: object
                  properties:
                    id:
                      type: string
                    name:
                      type: string
                    summary:
                      type: string
"""

In [ ]:
%store openapi_yaml >extension-api/extension.yaml

## Test with LangChain OpenAPI Endpoints


**TODO** do we want to keep this?

### Basic LLM Prompt to test setup

In [ ]:
# todo fix versions + code
import os

from langchain import PromptTemplate, LLMChain
from langchain.llms import VertexAI
from langchain.tools import OpenAPISpec, APIOperation
from langchain.chains import OpenAPIEndpointChain
from langchain.requests import Requests

In [ ]:

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What companies are in the 'Electronic Products Manufacturers' industry?"

llm_chain.run(question)

### Test with OpenAPIEndpointChain

In [ ]:
spec = OpenAPISpec.from_file("extension-api/extension.yaml")
operation = APIOperation.from_openapi_spec(spec, "/companies_in_industry", "get")
chain = OpenAPIEndpointChain.from_api_operation(
    operation,
    llm,
#    requests=Requests(),
    verbose=True,
    return_intermediate_steps=True,  # Return request and response text
)

output = chain("What companies are in the 'Electronic Products Manufacturers' industry?")

In [ ]:
output

### Import libraries

In [ ]:
from google.cloud.aiplatform.private_preview.vertex_agents.app import App, Session
from google.cloud.aiplatform.private_preview.vertex_agents.agent import Agent
from vertexai.preview.extensions import Extension


#Vertex AI Agent Applications

##Create App

There are two options for creating an app.


1.   use the App.create function, which will create a new app for you
2.   initialize a Vertex Agents App object with the resource name of a previously created App


Here we create our **Company News** GenAI application to manage our agents and serve as entry point for our user sessions.



In [ ]:
app = App.create(display_name='Company News App',
                 description='An application to inform people about company news for specific industries and people working in those companies')



# To load an existing app
# app = App("projects/<project_id>/locations/<location_id>/apps/<app_id>")

##List Apps

You can list all the apps in a project, which will return a list of operational Apps.

Alternatively, you can get a user-friendly mapping of App display names and resource names.

In [ ]:
all_apps = App.list_apps()

for app in all_apps:
  print(app.display_name, app.description, app.app_name)

# all_apps

Company News App An application to inform people about company news for specific industries and people working in those companies projects/990868019953/locations/us-central1/apps/6543167308615909376
Company News App An application to inform people about company news for specific industries and people working in those companies projects/990868019953/locations/us-central1/apps/1931481290188521472
Company News App An application to inform people about company news for specific industries and people working in those companies projects/990868019953/locations/us-central1/apps/2607021234294095872
Company News App An application to inform people about company news for specific industries and people working in those companies projects/990868019953/locations/us-central1/apps/517351007194185728
Default App 2510dfb5-d450-4a9d-ac20-ead9d9387a21 Default Description projects/990868019953/locations/us-central1/apps/6354016124266348544
Company News App An application to inform people about company news

# Agent Tools

There are two type of tools for your Agent, _Function Call_ tools that provide a signature and

###Function Call Tools

In order for your agent to call a function, you need to provide some information about the function to call. You can use the FunctionDeclaration class to construct the function declaration you wish to use, and add this to your app when you create or update.

In [ ]:
from vertexai.generative_models import FunctionDeclaration, Part

function_declaration = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters={
        "type": "OBJECT",
        "properties": {
            "location": {
                "type": "STRING",
                "description": "The city and state, e.g. San Francisco, CA"
            },
            "unit": {
                "type": "STRING",
                "enum": [
                    "celsius",
                    "fahrenheit",
                ]
            },
        },
        "required": [
            "location"
        ]
    },
)

###Extension Tools

You can integrate both first party extensions from Google and third party (like our) services as Extension tools for your agent.

More details can be found in https://cloud.google.com/vertex-ai/generative-ai/docs/extensions/create-extension.

#### First Party Extension from Google


To get a reference to a first party extension, you can use the `Extensions.from_hub(...)` method.


In [ ]:
code_interpreter_extension = Extension.from_hub("code_interpreter")

INFO:vertexai.extensions._extensions:Creating Extension
INFO:vertexai.extensions._extensions:Create Extension backing LRO: projects/990868019953/locations/us-central1/extensions/3003214856200388608/operations/6524992553207463936
INFO:vertexai.extensions._extensions:Extension created. Resource name: projects/990868019953/locations/us-central1/extensions/3003214856200388608
INFO:vertexai.extensions._extensions:To use this Extension in another session:
INFO:vertexai.extensions._extensions:extension = vertexai.preview.extensions.Extension('projects/990868019953/locations/us-central1/extensions/3003214856200388608')


#### Third party

To create a 3p extension, you will need to specify the display name, description, and manifest of your extension. You can specify the Open API spec in YAML string format, and set the `open_api_yaml`, or you can upload the YAML file to a Google Cloud Storage bucket, and reference it in `open_api_gcs_uri`.

We're just using our OpenAPI spec from before in the `openapi_yaml` variable.


Here we create our `Company News` extension with a description, and manifest holding the OpenAPI spec, and no auth because we're using a public dataset.

In [ ]:
company_news_extension = Extension.create(
    display_name = "Company News",
    description = "Service for company news information, industries, articles, people at the companies and more.",
    manifest = {
        "name": "company_news",
        "description": "Company News Extension",
        "api_spec": {
            "open_api_yaml": openapi_yaml,
        },
        "auth_config": {
            "auth_type": "NO_AUTH",
        },
    },
)

INFO:vertexai.extensions._extensions:Creating Extension
INFO:vertexai.extensions._extensions:Create Extension backing LRO: projects/990868019953/locations/us-central1/extensions/1224293003389042688/operations/469902854207832064
INFO:vertexai.extensions._extensions:Extension created. Resource name: projects/990868019953/locations/us-central1/extensions/1224293003389042688
INFO:vertexai.extensions._extensions:To use this Extension in another session:
INFO:vertexai.extensions._extensions:extension = vertexai.preview.extensions.Extension('projects/990868019953/locations/us-central1/extensions/1224293003389042688')


### Checking and cleaning Extensions

with `Extension.list()` you can see the currently registered extensions in your account.

And with `extension.delete()` you can remove old ones that you don't need anymore.


In [ ]:
Extension.list()

 resource name: projects/990868019953/locations/us-central1/extensions/3003214856200388608,
 resource name: projects/990868019953/locations/us-central1/extensions/2646304585731276800]

In [ ]:
# for e in Extension.list():
#  e.delete()

# Adding Agents to our Application

### Agents

Agents exist under an App. Agents can perform small, specific tasks utilizing Vertex Extensions and function calling. Agents require a display name, and a set of instructions.

When creating an agent, you must specify the following:


1.   Display Name
2.   Instructions - the detailed instructions the Agent should follow. This is where you instruct your agent when to call your functions / extensions and with what parameters.

You also have the option to specify the extensions and functions you will be using. Please note that if you reference an extension or function in the instructions, you ***must*** include an extension/function with the same display name in the extensions/functions lists.

### Create Agent
You can create an agent for your app by using the add_agent method. When creating an agent, you must specify all the required fields discussed above.



In [ ]:
DISPLAY_NAME = "Company News Agent"
INSTRUCTIONS = "You are an expert in company information and news. You have access to sources like articles, companies and the people working there to answer user questions."

created_agent = app.add_agent(display_name=DISPLAY_NAME,
                              instructions=INSTRUCTIONS)

### Create a single Agent App

If you want to create an App and Agent in a single turn (i.e. avoiding using create_app -> add_agent) you can use `App.create_single_agent_app`. You must specify the agent display name, and then optionally specify tools for the agent, and the App's display name and description. This function returns an operable App.


**TODO** which of the two approaches to use?

In [ ]:
%skip
DISPLAY_NAME = "Company News Agent"
INSTRUCTIONS = "You are an expert in company information and news. You have access to sources like articles, companies and the people working there to answer user questions."
app = App.create_single_agent_app(agent_display_name=DISPLAY_NAME,
                                   instructions=INSTRUCTIONS)

UsageError: Line magic function `%skip` not found.


##Update Agent

You can update any and all of the fields of an Agent by specifying them to the Agent.update function. Any fields that aren't specified will not be updated.


In [ ]:
# app.list_agents()

#### Instructions

TODO

In [ ]:
INSTRUCTIONS = """
You are an expert in company information and news.
You have access to sources like articles, companies and the people working there to answer user questions.
If a user asks about companies in an industry, find the correct industry name first from the list and the use that to gather the companies.
If a user asks about articles in a specific month provide the date as yyyy-mm-dd format.
Companies mentioned in articles can be retrieved by the article id.
You can get detail for a company by company id or deatils for an article with the article id.
Also the people working for a company can be retrieved via the company id.

If the user asks you to compute a numeric value or analyse data use the code interpreter.

Only respond based on the response of the tools. Do not create your own answers.
"""

updated_agent = created_agent.update(
    new_instructions=INSTRUCTIONS,
    new_model="projects/{}/locations/us-central1/publishers/google/models/gemini-1.5-pro".format(PROJECT_ID),
#    new_functions=[function_declaration],
    new_extensions={
#        'Code Interpreter': code_interpreter_extension,
        'Company News Extension': company_news_extension
    }
)

In [ ]:
updated_agent

##List Agents

You can list all the agents exiting under an App. This will return operatable Agent objects.

In [ ]:
app.list_agents()

[
 agent_name: projects/990868019953/locations/us-central1/apps/2607021234294095872/agents/7564231530529161216
 display_name: Company News Agent
 model: projects/vertex-ai-neo4j-extension/locations/us-central1/publishers/google/models/gemini-1.5-pro
 instructions: 
 You are an expert in company information and news.
 You have access to sources like articles, companies and the people working there to answer user questions.
 If a user asks about companies in an industry, find the correct industry name first from the list and the use that to gather the companies.
 If a user asks about articles in a specific month provide the date as yyyy-mm-dd format.
 Companies mentioned in articles can be retrieved by the article id.
 You can get detail for a company by company id or deatils for an article with the article id.
 Also the people working for a company can be retrieved via the company id.
 
 If the user asks you to compute a numeric value or analyse data use the code interpreter.
 
 Only re

##Get Agent

You can also get a specific Agent using either the resource name or the display name

In [ ]:
agent = app.get_agent('Company News Agent')

In [ ]:
agent


agent_name: projects/990868019953/locations/us-central1/apps/2607021234294095872/agents/7564231530529161216
display_name: Company News Agent
model: projects/vertex-ai-neo4j-extension/locations/us-central1/publishers/google/models/gemini-1.5-pro
instructions: 
You are an expert in company information and news.
You have access to sources like articles, companies and the people working there to answer user questions.
If a user asks about companies in an industry, find the correct industry name first from the list and the use that to gather the companies.
If a user asks about articles in a specific month provide the date as yyyy-mm-dd format.
Companies mentioned in articles can be retrieved by the article id.
You can get detail for a company by company id or deatils for an article with the article id.
Also the people working for a company can be retrieved via the company id.

If the user asks you to compute a numeric value or analyse data use the code interpreter.

Only respond based on t

Alternatively, you can get an Agent using the Agent class

Note that the only way to create a new Agent is to create it under an App, using `app.add_agent()`. The Agents constructor can only be used to get a reference to an Agent that has already been created.

In [ ]:
# agent2 = Agent(agent_id='projects/<project id>/locations/<location id>/<app id>/agents/<agent id>')

## Delete Agent
Using either the display name or the fully qualified resource name, you can delete a specific agent under the App.

In [ ]:
# app.delete_agent('<Agent Name>')

#Session

NOTE: you can remove `.content` from the end of each run call to get more detailed information about each session action

In [ ]:
session = app.start_session()

INFO:google.cloud.aiplatform.private_preview.vertex_agents.app:Session init
INFO:google.cloud.aiplatform.private_preview.vertex_agents.app:Session init 2
INFO:google.cloud.aiplatform.private_preview.vertex_agents.app:app name = projects/990868019953/locations/us-central1/apps/2607021234294095872
INFO:google.cloud.aiplatform.private_preview.vertex_agents.app:Creating session without request


## Multi-turn queries without tool calls


In [ ]:
result = session.run("Give me 5 industry names")
result.content

role: "model"
parts {
  text: "Here are 5 industry names: Electronic Products Manufacturers, Enterprise Software Companies, Computer Hardware Companies, Business Software Companies, Mobile Phone Manufacturers. \n"
}

In [ ]:
session.get_history()

name: "projects/990868019953/locations/us-central1/apps/2607021234294095872/sessions/6106191151167963136"
actions {
  message {
    role: "user"
    parts {
      text: "Give me 5 industry names"
    }
  }
  create_time {
    seconds: 1724916911
    nanos: 695676000
  }
  turn: 1
}
actions {
  tool_use {
    extension_invocation {
      extension_name: "projects/990868019953/locations/us-central1/extensions/1224293003389042688"
      operation_id: "industries"
    }
    input_parameters {
    }
    output_parameters {
      fields {
        key: "output"
        value {
          list_value {
            values {
              string_value: "Electronic Products Manufacturers"
            }
            values {
              string_value: "Enterprise Software Companies"
            }
            values {
              string_value: "Computer Hardware Companies"
            }
            values {
              string_value: "Business Software Companies"
            }
            values {

## User query that invokes client side function


In [ ]:
session.run("what's the weather in San Francisco in fahrenheit").content

role: "model"
parts {
  text: "I cannot provide that information. I can only access company information and news. \n"
}

### Feed function response

The user passes back the client side function execution result in the function response back to Agent

In [ ]:
session.run(Part.from_function_response(name="get_current_weather", response={"temperature": "65"})).content

role: "model"
parts {
  text: "The temperature in San Francisco is 65 degrees fahrenheit. \n"
}

## User query that invokes custom extension


In [ ]:
session.run("what are 3 companies in the 'Computer Hardware' space in California and their ids")

session: "projects/990868019953/locations/us-central1/apps/517351007194185728/sessions/2034937088025034752"
content {
  role: "model"
  parts {
    text: "Here are 3 companies in the \'Computer Hardware\' space that are located in California:\n\n* Apigee (EAHSCRG21ODCH1xk3354mxA) is a Software company based in San Jose, California. \n* Xactly (E3lZVAk_KOpeQsRX4AS6F8w) is a Software company based in San Jose, California.\n* Aerohive Networks (Ec2-e0bEXPuOQvE1KyUfLJg) is a Software company based in Milpitas, California. \n"
  }
}
actions {
  message {
    role: "user"
    parts {
      text: "what are 3 companies in the \'Computer Hardware\' space in California"
    }
  }
  create_time {
    seconds: 1724847506
    nanos: 543385000
  }
  turn: 3
}
actions {
  tool_use {
    extension_invocation {
      extension_name: "projects/990868019953/locations/us-central1/extensions/3429930920893743104"
      operation_id: "companies_in_industry"
    }
    input_parameters {
      fields {
       

In [ ]:
result = session.run("What people are working at these companies in which roles?")
result.content

role: "model"
parts {
  text: "Here are the people associated with those companies:\n\n* **Apigee** (EAHSCRG21ODCH1xk3354mxA): \n    * Promod Haque - BOARD_MEMBER\n    * Bob L Corey - BOARD_MEMBER\n    * Neal Dempsey - BOARD_MEMBER\n* **Xactly** (E3lZVAk_KOpeQsRX4AS6F8w):\n    * Chris Cabrera - BOARD_MEMBER\n    * Jeff Wilson - BOARD_MEMBER\n    * Brian Sheth - BOARD_MEMBER\n    * Betty Hung - BOARD_MEMBER\n    * Dave Pidwell - BOARD_MEMBER\n    * Chris Cabrera - CEO\n* **Aerohive Networks** (Ec2-e0bEXPuOQvE1KyUfLJg): \n    * David K Flynn - CEO \n"
}

In [ ]:
result = session.run("What 3 articles in January 2021 had a really good sentiment?")
result.content

role: "model"
parts {
  text: "The top 3 articles from January 2021 with the highest sentiment scores are:\n\n1. **Cigniti signs up as a Medecision Liberation Strategic Partner** (Sentiment: 0.967)\n2. **Ubergizmo’s Best of CES 2021** (Sentiment: 0.912) \n3. **IBM acquires Salesforce consultancy 7Summits** (Sentiment: 0.939) \n"
}

In [ ]:
result = session.run("Can you summarize the text content of these articles?")
result.content

role: "model"
parts {
  text: "Cigniti signs up as a Medecision Liberation Strategic Partner: This article discusses Cigniti Technologies\' partnership with Medecision, a health management company. Cigniti\'s quality engineering and testing services will help ensure a seamless experience for healthcare customers using Medecision\'s solutions. \n\nUbergizmo’s Best of CES 2021: This article highlights Ubergizmo\'s top picks from CES 2021, showcasing innovative products like the Samsung Galaxy S21 series, Acer Predator Triton 300 SE gaming laptop, and Qualcomm\'s 3D Sonic Sensor Gen 2. \n\nIBM acquires Salesforce consultancy 7Summits: This article reports on IBM\'s acquisition of 7Summits, a Salesforce consultancy firm. The acquisition aims to strengthen IBM\'s Salesforce services and experience design capabilities, further supporting clients\' digital transformations. \n"
}

In [ ]:
result

session: "projects/990868019953/locations/us-central1/apps/517351007194185728/sessions/4340780097238728704"
content {
  role: "model"
  parts {
    text: "Cigniti signs up as a Medecision Liberation Strategic Partner: This article discusses Cigniti Technologies\' partnership with Medecision, a health management company. Cigniti\'s quality engineering and testing services will help ensure a seamless experience for healthcare customers using Medecision\'s solutions. \n\nUbergizmo’s Best of CES 2021: This article highlights Ubergizmo\'s top picks from CES 2021, showcasing innovative products like the Samsung Galaxy S21 series, Acer Predator Triton 300 SE gaming laptop, and Qualcomm\'s 3D Sonic Sensor Gen 2. \n\nIBM acquires Salesforce consultancy 7Summits: This article reports on IBM\'s acquisition of 7Summits, a Salesforce consultancy firm. The acquisition aims to strengthen IBM\'s Salesforce services and experience design capabilities, further supporting clients\' digital transformation

In [ ]:
result = session.run("Which companies were mentioned by these articles?")
result.content

role: "model"
parts {
  text: "The articles mentioned these companies: Medecision, Samsung, Klaxoon, 7Summits, IBM Global Services, MuleSoft, and IBM. \n"
}

In [ ]:
result = session.run("And who are the people that work there in which roles?")
result.content

role: "model"
parts {
  text: "Here are the people and their roles at the companies you mentioned:\n\n**Medecision:**\n\n* David St Clair - BOARD_MEMBER\n* Arun Prasad - BOARD_MEMBER\n* Frank Adams - BOARD_MEMBER\n* Ken Young - CEO\n\n**Samsung:**\n\n* LEE JAE CHUL 이재철 - BOARD_MEMBER\n* JAWOOK KOO - CEO \n\n**Klaxoon:** \n\n* No people found.\n\n**7Summits:** \n\n* Matt Garratt - INVESTOR\n* Paul Stillmank - CEO \n\n**IBM Global Services:** \n\n* No people found.\n\n**MuleSoft:** \n\n* Sarah Dods - BOARD_MEMBER\n* Joachim Wettermark - BOARD_MEMBER\n* Yee Darryl - BOARD_MEMBER\n* Greg Schott - BOARD_MEMBER\n* Simon Parmett - CEO \n\n**IBM:** \n\n* Arvind Krishna - BOARD_MEMBER & CEO\n* Bill Kelleher - BOARD_MEMBER\n* Michael L. Eskew - BOARD_MEMBER\n* Mark Ritter - BOARD_MEMBER\n* George Eapen - BOARD_MEMBER\n* Christina Montgomery - BOARD_MEMBER\n* Michelle Howard - BOARD_MEMBER\n* Al Zollar - BOARD_MEMBER\n* Ginni Rometty - BOARD_MEMBER\n* Thomas Buberl - BOARD_MEMBER\n* William McNab

## User query that invokes Google extension (Code interpreter)

In [ ]:
question="""
Write a function to compute the next state of a game of life board represented by
a list of live cell x,y coordinates ([0,0] is bottom left), and run it with the parameter [[0,0],[1,0],[2,0],[2,1],[1,2]].
Show the output of that function call and the code."
"""

session.run(question).content

role: "model"
parts {
  text: "```python\ndef game_of_life_step(live_cells):\n    \"\"\"\n    Computes the next state of a Game of Life board.\n\n    Args:\n        live_cells: A list of [x, y] coordinates representing live cells.\n\n    Returns:\n        A list of [x, y] coordinates representing live cells in the next state.\n    \"\"\"\n\n    def is_live_cell(x, y):\n        return [x, y] in live_cells\n\n    def count_live_neighbors(x, y):\n        count = 0\n        for dx in [-1, 0, 1]:\n            for dy in [-1, 0, 1]:\n                if (dx != 0 or dy != 0) and is_live_cell(x + dx, y + dy):\n                    count += 1\n        return count\n\n    next_state = []\n    for x in range(-2, 5):\n        for y in range(-2, 5):\n            neighbors = count_live_neighbors(x, y)\n            if (is_live_cell(x, y) and 2 <= neighbors <= 3) or (not is_live_cell(x, y) and neighbors == 3):\n                next_state.append([x, y])\n\n    return next_state\n\ninitial_state = [[0, 0],

## Get conversation history

You can view the entire conversation history by using `session.get_history()`

In [ ]:
session.get_history()

Alternatively, you can list all the sessions your app has run

In [ ]:
sessions = app.list_sessions()

In [ ]:
print(sessions[0])

## Cleanup

* delete session (optinally)
* delete agent (optionally)
* delete application

In [ ]:
# Deleting an Agent by Name

# app.delete_agent('Company News Agent')

In [ ]:
# delete extensions
for e in Extension.list():
  e.delete()

In [ ]:
# app.delete_session('<Session Name>')

In [ ]:
App.delete(app_name="projects/990868019953/locations/us-central1/apps/3266798579703873536")